In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='3'
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# from transformers import PaliGemmaProcessor, PaliGemmaForConditionalGeneration
from src.processing_paligemma import PaliGemmaProcessor
from src.modeling_paligemma import PaliGemmaForConditionalGeneration
import torch
from torch.utils.data import DataLoader
from src.data import ClevrerDataset
import torchvision.transforms as transforms
import numpy as np

# --- Setup ---
model_id = "google/paligemma2-3b-mix-448"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
activations_dir = "../extracted_activations"  # Directory to save activations
os.makedirs(activations_dir, exist_ok=True)

# Load model and processor
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id).to(device).eval()
processor = PaliGemmaProcessor.from_pretrained(model_id)

# --- Load a small subset of the CLEVRER dataset for demonstration ---
frames_root = "../frame_captures/"  # Replace with the actual path to your CLEVRER frames
json_path = "../train.json"  # Replace with the actual path to your CLEVRER annotations

# Define transforms (adjust as needed)
transform = transforms.Compose([
    transforms.Resize((448, 448)),
    transforms.ToTensor(),
])

# Load the full dataset
full_dataset = ClevrerDataset(frames_root=frames_root, json_path=json_path, transform=transform)



/data1/ai24mtech02001/miniconda3/envs/physic/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.08it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
# Create a small subset (e.g., first sample) for demonstration
subset_indices = [9]

print(subset_indices)

subset_dataset = torch.utils.data.Subset(full_dataset, subset_indices)
dataloader = DataLoader(subset_dataset, batch_size=1, shuffle=False)

item = next(iter(dataloader))


[9]


In [8]:
question = "<image> "+item["question"][0]

In [9]:

model_inputs = processor(text=question, images=[item["frames"]], return_tensors="pt").to(torch.bfloat16).to(model.device)

ValueError: axes don't match array

In [4]:
out = model(**model_inputs)

******************************
Image features shape: torch.Size([8, 128, 2304])
Input embeds shape: torch.Size([1, 8204, 2304])


/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:371: masked_scatter_size_check: block: [0,0,0], thread: [0,0,0] Assertion `totalElements <= srcSize` failed.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [8]:
import importlib
importlib.reload(importlib.import_module('src.activation_manager'))
from src.activation_manager import ActivationReplacementExperiment

# --- Initialize ActivationReplacementExperiment ---
activation_experiment = ActivationReplacementExperiment(model, processor, device=device, activations_dir=activations_dir)


In [4]:
# --- Define the layer(s) to extract activations from ---
layer_names_to_extract = [
        # 'vision_tower.vision_model.encoder.layers.5.self_attn',
        # 'vision_tower.vision_model.encoder.layers.4.self_attn.k_proj',
        'language_model.model.layers.15.self_attn.q_proj',
        'language_model.model.layers.15.self_attn.k_proj',
        'language_model.model.layers.15.self_attn.v_proj',
           ]


# --- Extract 'real' activations ---
activation_experiment._extract_activations(dataloader, layer_names_to_extract, store_as='real')

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


ValueError: axes don't match array

: 

In [ ]:
# --- Save the extracted activations to a text file ---
output_file = os.path.join(activations_dir, "real_activations.txt")

for layer_name in layer_names_to_extract:  # Iterate only through the layers we extracted
        if layer_name in activation_experiment.real_activations:
            activations = activation_experiment.real_activations[layer_name]
            # activations = activations.cpu().numpy().tolist()
            print(activations.shape)


# with open(output_file, "w") as f:
#     for layer_name in layer_names_to_extract:  # Iterate only through the layers we extracted
#         if layer_name in activation_experiment.real_activations:
#             f.write(f"Layer: {layer_name}\n")
#             activations = activation_experiment.real_activations[layer_name]
#             activations = activations.cpu().numpy().tolist()
#             f.write(f"    {activations}\n")
#             f.write("\n")

# print(f"Extracted activations from specified layers saved to: {output_file}")

torch.Size([1, 8204, 2048])
torch.Size([1, 8204, 1024])
torch.Size([1, 8204, 1024])
